In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
### Dropout ###
DRPT_prob = 0.5
###############

In [ ]:
c1_out_channels = K = 96  ## AKA K
no_of_RCL_blocks = 4

In [ ]:
class RCLBlock(nn.Module):
    def __init__(self):
        super(RCLBlock, self).__init__()

        self.feedforward_filter = nn.Conv3d(K,K,(1,3,3),stride=(2,1,1),padding=(0,1,1))
        self.recurrent_filter = nn.Conv3d(K,K,(1,3,3),stride=(2,1,1),padding=(0,1,1))
        self.timesteps = 3

    def forward(self, input):
        out = feedforward_output = (F.relu(self.feedforward_filter(input)))
        for i in range(self.timesteps):
            out = self.recurrent_filter(out)
            out = out + feedforward_output
            out = (F.relu(out))
        return out

In [ ]:
class RCNN(nn.Module):
    def __init__(self, n_classes=6):
        super(RCNN, self).__init__()
        
#         self.LRN = nn.LocalResponseNorm(LRN_size,alpha=LRN_alpha)
        self.n_frames = 128
#         self.Dropout = nn.Dropout(DRPT_prob)
        self.MaxPool = nn.MaxPool3d(3,stride=(2,2,2),padding=1)
        self.conv1 = nn.Conv3d(self.n_frames,K,(3,2,2),padding=(0,0,0))
        self.ReLU = nn.ReLU()
#         self.conv2 = nn.Conv2d(K,64,5)
        self.RCL1 = RCLBlock()
        self.RCL2 = RCLBlock()
        self.RCL3 = RCLBlock()
        self.RCL4 = RCLBlock()
        self.RCL5 = RCLBlock()
        self.RCL6 = RCLBlock()

        self.Linear = nn.Linear(K,n_classes)

    


    def forward(self, x):
        out = self.conv1(x)
#         print(out.shape)
        # return out
        out = self.MaxPool(out)
        # return out
         ## RCL Block :- 1
        out = self.RCL1(out)
#         out = self.Dropout(out)
         ## RCL Block :- 2
        out = self.RCL2(out)
        out = self.MaxPool(out)

#         out = self.Dropout(out)
        
        ## RCL Block :- 3
        out = self.RCL3(out)
#         out = self.Dropout(out)
         ## RCL Block :- 4
        out = self.RCL4(out)
        
#         out = self.conv2(out)
        out = self.MaxPool(out)
            
#         out = self.Dropout(out)
         ## RCL Block :- 4
        out = self.RCL5(out)
#         out = self.Dropout(out)
#          ## RCL Block :- 4
        out = self.RCL6(out)

        ## Global Max Pooling
        out = F.max_pool3d(out, out.size()[2:])  ## after this, out.shape == N,K,1,1
        out = out.view(out.size()[0],out.size()[1])  ##  after this, out.shape == N,K  
        out = self.Linear(out)  ## after this, out.shape == N,n_classes
        out = F.softmax(out,dim=1)
        return out

In [ ]:
from ucf_data import UCF_dataloader

In [ ]:
inp,labels = next(iter(UCF_dataloader))

In [ ]:
oo = net(inp.float().cuda())

In [ ]:
predicted_labels = torch.argmax(oo, dim=1)
torch.eq(predicted_labels.cpu(),labels).cpu().sum().view(1).numpy()[0]

In [ ]:
net = RCNN()

In [ ]:
net

In [ ]:
sum([param.nelement() for param in net.parameters()])

In [ ]:
learning_rate = 1e-5
epoch = 30

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(), lr=learning_rate)

In [ ]:
use_gpu = torch.cuda.is_available() #and False

if use_gpu:
    net = net.cuda()

In [ ]:
# net = net.cpu()

In [ ]:
loss_trend = []
accuracy_trend = []

In [ ]:
for i in range(epoch):
    loss_each_epoch = []
    running_accuracy = []
    mini_count = 1
    for images,labels in UCF_dataloader:
        net = net.train()
        if use_gpu:
            images = images.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()
        # scheduler.optimizer.zero_grad()
        output = net(images.float())
        # print(output)
        # print(target)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        # scheduler.step(loss)
        loss_to_append = loss.clone().cpu().view(1).data.numpy()[0]
        print("Epoch : {}, Mini-Epoch : {}, Loss: {}".format(i+1,mini_count,loss_to_append))
        mini_count += 1
        loss_each_epoch.append(loss_to_append)

    loss_trend.append(sum(loss_each_epoch))


    print('##### Epoch {} #####'.format(i+1))
    print('Loss : {}'.format(sum(loss_each_epoch)))
    print('####################')


In [ ]:
state = {
    'epoch': 30,
    'state_dict': net.state_dict(),
    'optimizer': optimizer.state_dict(),
}

In [ ]:
torch.save(state,'23eps_4RCLs_3 ts_trained.net')

In [ ]:
loss_trend

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(loss_trend)

In [ ]:
fig = plt.figure()
fig.suptitle('RCNN-96-Vid-4RCL-3-timesteps | Min loss:- 23.4908', fontsize=12)
ax = fig.add_subplot(111)
fig.subplots_adjust(top=0.85)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.plot(loss_trend)
fig.savefig('loss-RCNN-96-Vid-4RCL-3-timesteps.png')

In [ ]:
min(loss_trend)